# Расчёт времени нахождения в терапевтическом диапазоне методом Розенталя

**Описание проекта:** <br>
    Пациенты, перенесшие операции на сердце принимают кроверазжижающие препараты на основе варфарина. Варфарин очень опасное вещество, так как его передозировка может вызывать кровотечения как наружние, так и внутренние. Если же пациент принимает слишклм малую дозу то возникает риск образования тромбов. Параметром, который позволяет контролированть правильность дозировки, является МНО (международное нормализованное отношение). У каждого пациента, в зависимости от его диагноза, должен быть свой терапевтический диапазон в котором должен находится этот параметр, плюс врач может его скорректировать по конкретного пациента, исходя из его истории лечения. У разных людей встречается разная чувствительность к препаратам, содержащим варфарин, поэтому дозировка для каждого пациента подбирается индивидуально на основе его предыдущих дозировок и значений МНО. МНО сдаётся один раз в 10-20 дней и может как попадать в целевой диапазон, так и выпадать из него. Критерием эффективности назначаемой дозировки является такой параметр, как TTR (Time in Therapeutic Range). Проблема заключается в том, что нет утвержденных стандартов его расчёта.
    
**Цель:**
   1. Изучить данные и составить алгоритм расчёта TTR метом Розенталя.
   2. Учесть, что у пациента может быть несколько заболеваний, и несколько соответствующих им целевых диапазонов.
   3. Иметь в виду, что врач может внести изменения в терапевтический диапазон.
   4. Рассчитать TTR как отношение веремени пребывания в индивидуалльном целевом диапазоне к общему времени лечения.
   5. Сгруппировать результаты по годам отдельно для пациентов, сдающих анализ дома, и для пациентов с собственным прибором.
   6. Вывести график.
   
**Этапы исследования:**

   1. Загрузка данных и изучение общей информации;
   2. Анализ данных;
   3. Добавление новых признаков;
   4. Расчёт;
   5. Презентация результатов.

**Импортируем библиотеки:**

In [43]:
import pandas as pd 
import plotly.express as px

**Загружаем данные:**

In [44]:
inr = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_inr.csv')
indications = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_indications.csv')
treatment_data = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_treatment_data.csv')
patient_institutions = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_patient_institutions.csv')

**Посмотрим, из чего состоит таблица inr:**

In [45]:
inr.head() # For TTR calculation

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...
2,6026394360,6059271230,2024-04-12 00:00:00,3.1,LAB,CSP,2.0,6140543514,2024-04-18 08:21:25.910798,return ''.sprintf(gettext('Huidige INR %s ligt...
3,6026394638,6059368614,2023-11-24 11:42:00,3.0,ZMA,CSP,2.0,6137177792,2023-11-24 11:42:44.286527,return ''.gettext('Er kon geen doseervoorstel ...
4,6026138194,6026318204,2018-03-23 00:00:00,4.4,LAB,CSP,3.0,6026326660,2018-03-23 11:00:54.859975,return ''.gettext('Er zijn minder dan 2 opeenv...


**Здесь все значения МНО с датой, временем и типом терапевтического диапазона, назначаемого автоматически, но нет информации об индивидуальном терапевтическом диапазоне.**

**Посмотрим, из чего состоит таблица indications:**

In [46]:
indications.head() # automatic therapeutic range can be taken from here

,peso_id,trbh_id,effective_from,effective_to,code,name,remark,type,location,target_range,id
0,6026136398,6026185844,2016-11-28 00:00:00,NaN,134,Onstabiele angina pectoris,NaN,NaN,NaN,2,6026185868
1,6026137016,6026218258,2017-03-10 00:00:00,NaN,1049,Mechanische mitralis hartklepprothese + Risico...,NaN,мех,мк,2,6057488560
2,6026137406,6026250702,2017-08-15 00:00:00,NaN,1021,Bioklepprothese + Risicofactor,NaN,NaN,NaN,1,6055527868
3,6026137934,6026292136,2022-10-23 00:00:00,NaN,1029,Overige (zeldzame) indicaties (streefgebied 1),Протезирование ВОА,NaN,NaN,1,6123427048
4,6026138020,6026301532,2018-01-22 00:00:00,NaN,1040,AF + (cerebrale) embolie,NaN,NaN,NaN,1,6035999650


**Здесь указан автоматический терапевтический диапазон для каждого пациента. Важно, что у него есть время действия. Для одного пациента может существовать несколько строк.**

**Можем сразу найти максимальный целевой диапазон у пациента, так как учитывается именно он:**

In [47]:
indications.groupby('peso_id')['target_range'].max() # maximum automatic therapeutic range

peso_id
6026136398    2
6026136402    1
6026136420    2
6026136424    2
6026136428    2
             ..
6028057454    2
6028057456    2
6028057458    1
6028057472    1
6028057474    1
Name: target_range, Length: 9272, dtype: int64

**Посмотрим, из чего состоит таблица treatment_data:**

In [48]:
treatment_data.info() # personal target range is here

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9463 entries, 0 to 9462
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   peso_id                        9463 non-null   int64  
 1   trbh_id                        9463 non-null   int64  
 2   effective_from                 9463 non-null   object 
 3   effective_to                   2683 non-null   object 
 4   duration_days                  90 non-null     float64
 5   duration_auto_stop             9463 non-null   object 
 6   status                         9463 non-null   object 
 7   dosing_max_period              9463 non-null   int64  
 8   dosing_margin_days             9463 non-null   int64  
 9   target_range_deviation_lower   1434 non-null   float64
 10  target_range_deviation_upper   1434 non-null   float64
 11  target_range_deviation_reason  1434 non-null   object 
 12  curr_status_selfmeasurement    9463 non-null   o

**В этом файле появляется информация об изменениях целевого диапазона МНО, внесенного врачом**

**Отфильтруем пациентов с измененным целевым диапазоном:**

In [49]:
treatment_data.query('target_range_deviation_lower.notna()') # the list of patients with not empty personal target range

,peso_id,trbh_id,effective_from,effective_to,duration_days,duration_auto_stop,status,dosing_max_period,dosing_margin_days,target_range_deviation_lower,...,curr_status_selfdosing,curr_status_regular,curr_status_training,curr_status_home_poli,curr_status_dosing_dr,curr_status_new_patient,permanent_message,puncture_list_message,call_list_message,reason
6,6026393996,6059129490,2020-04-27 00:00:00,2021-09-03 00:00:00,NaN,N,COMPLETED,30,10,2.5,...,N,N,N,N,NaN,NaN,"пластика ТК, стентирование левой ПА в 2014 году",NaN,NaN,return ''.'дубль';
21,6026351208,6049591760,2019-11-18 00:00:00,2021-09-03 00:00:00,NaN,N,COMPLETED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'дубль';
23,6026138730,6026365748,2018-06-21 00:00:00,2018-08-13 00:00:00,NaN,N,DIED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'Просто умер';
40,6026136880,6026209870,2017-02-03 00:00:00,2019-09-07 00:00:00,NaN,N,COMPLETED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return gettext('Foutief ingevoerd').'';
42,6026299230,6043411018,2019-10-06 00:00:00,2023-05-13 00:00:00,184.0,J,COMPLETED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'кардиолог отменил варфарин';
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9410,6026137296,6026243432,2017-07-19 00:00:00,NaN,NaN,N,ACTIVE,30,10,2.0,...,N,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9442,6027962856,6116736742,2022-12-12 00:00:00,NaN,NaN,N,ACTIVE,30,10,1.5,...,N,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9446,6026137302,6026243758,2017-07-20 00:00:00,NaN,NaN,N,ACTIVE,30,10,2.5,...,N,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9455,6027926750,6103653352,2022-09-12 00:00:00,NaN,NaN,N,ACTIVE,30,10,2.5,...,N,N,NaN,N,NaN,N,"кардиолог МКДЦ (целевой уровень МНО 2,5-3,5)",NaN,NaN,NaN


**Объеденим информацию об автоматическом и ручном целевых диапазонах:**

In [50]:
ranges = treatment_data.merge(indications.groupby('peso_id')['target_range'].max(),on = 'peso_id', how = 'inner') # here we have as personal as automatic target range
ranges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9460 entries, 0 to 9459
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   peso_id                        9460 non-null   int64  
 1   trbh_id                        9460 non-null   int64  
 2   effective_from                 9460 non-null   object 
 3   effective_to                   2681 non-null   object 
 4   duration_days                  90 non-null     float64
 5   duration_auto_stop             9460 non-null   object 
 6   status                         9460 non-null   object 
 7   dosing_max_period              9460 non-null   int64  
 8   dosing_margin_days             9460 non-null   int64  
 9   target_range_deviation_lower   1434 non-null   float64
 10  target_range_deviation_upper   1434 non-null   float64
 11  target_range_deviation_reason  1434 non-null   object 
 12  curr_status_selfmeasurement    9460 non-null   o

In [51]:
ranges = ranges.query('effective_to.isna()').reset_index()

**Добавим новые столбцы и заполним их нулями для удобства:**

In [52]:
ranges['final_lower']= [0 for i in range (len(ranges))]
ranges['upper_lower']= [0 for i in range (len(ranges))]

**Посчитаем итоговый целевой диапазон для каждого пациента. Будем считать введенные врачом данные приоритетными:**

In [53]:
for i in range(len(ranges)): # here the final taget range is calculated
    if ranges.loc[i,'target_range_deviation_lower']>0:
        ranges.loc[i,'final_lower'] = ranges.loc[i,'target_range_deviation_lower']
        ranges.loc[i,'final_upper'] = ranges.loc[i,'target_range_deviation_upper']
    elif ranges.loc[i,'target_range'] == 2:
        ranges.loc[i,'final_lower'] = 2.5
        ranges.loc[i,'final_upper'] = 3.5
    elif ranges.loc[i,'target_range'] == 1:
        ranges.loc[i,'final_lower'] = 2.0
        ranges.loc[i,'final_upper'] = 3.0
final_ranges = ranges[['peso_id', 'target_range_deviation_lower', 'target_range_deviation_upper', 'target_range','final_lower','final_upper']]
final_ranges

,peso_id,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper
0,6026184960,NaN,NaN,1,2.0,3.0
1,6026198834,NaN,NaN,1,2.0,3.0
2,6026293414,NaN,NaN,1,2.0,3.0
3,6026202040,NaN,NaN,1,2.0,3.0
4,6026216114,NaN,NaN,1,2.0,3.0
...,...,...,...,...,...,...
6774,6027855788,NaN,NaN,1,2.0,3.0
6775,6026433462,2.5,3.0,1,2.5,3.0
6776,6026261980,NaN,NaN,2,2.5,3.5
6777,6028009272,NaN,NaN,1,2.0,3.0


In [54]:
final_ranges['peso_id'].nunique()

6778

**А теперь привяжем финальный целевой диапазон к таблице МНО:**

In [55]:
final_inr = inr.merge(final_ranges, on = 'peso_id') # All INRs with final target range
#final_inr['peso_id'].nunique()
final_inr

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,6028010260,6137309936,2024-02-09 00:00:00,1.0,ZMA,CSP,NaN,6139233834,NaN,NaN,NaN,NaN,1,2.0,3.0
139391,6028010260,6137309936,2023-11-30 00:00:00,1.0,ZMA,CSP,NaN,6137312894,NaN,NaN,NaN,NaN,1,2.0,3.0
139392,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5
139393,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5


**Добавим столбец с признаком 'входит/не входит в целевой диапазон':**

In [56]:
for i in range(len(final_inr)): # here the new column "inside" is filled in
    if final_inr.loc[i,'final_lower']<=final_inr.loc[i,'inr']<=final_inr.loc[i,'final_upper']:
        final_inr.loc[i,'inside'] = 100
    else:
        final_inr.loc[i,'inside'] = 0
final_inr        

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,6028010260,6137309936,2024-02-09 00:00:00,1.0,ZMA,CSP,NaN,6139233834,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0
139391,6028010260,6137309936,2023-11-30 00:00:00,1.0,ZMA,CSP,NaN,6137312894,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0
139392,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0
139393,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0


In [57]:
for i in range(len(final_inr)): # here the new column "below" is filled in
    if final_inr.loc[i,'final_lower']>final_inr.loc[i,'inr']:
        final_inr.loc[i,'below'] = 100
    else:
        final_inr.loc[i,'below'] = 0
final_inr   

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside,below
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0,0.0
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0,100.0
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0,0.0
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,6028010260,6137309936,2024-02-09 00:00:00,1.0,ZMA,CSP,NaN,6139233834,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0
139391,6028010260,6137309936,2023-11-30 00:00:00,1.0,ZMA,CSP,NaN,6137312894,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0
139392,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0,100.0
139393,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0,0.0


**Добавим столбец с признаком 'ниже/выше' целевого диапазона':**

In [58]:
for i in range(len(final_inr)): # here the new column "above" is filled in
    if final_inr.loc[i,'final_upper']<final_inr.loc[i,'inr']:
        final_inr.loc[i,'above'] = 100
    else:
        final_inr.loc[i,'above'] = 0
final_inr 

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside,below,above
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0,0.0,100.0
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0,100.0,0.0
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,6028010260,6137309936,2024-02-09 00:00:00,1.0,ZMA,CSP,NaN,6139233834,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0
139391,6028010260,6137309936,2023-11-30 00:00:00,1.0,ZMA,CSP,NaN,6137312894,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0
139392,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0,100.0,0.0
139393,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0,0.0,0.0


**Добавим столбец с годом взятия МНО для последующей группировки:**

In [59]:
final_inr['year'] = final_inr['inr_date'].astype("datetime64[ns]").dt.to_period("Y") #here the new column for year of INR is added
final_inr

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside,below,above,year
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0,0.0,100.0,2023
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0,100.0,0.0,2023
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2020
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2020
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,6028010260,6137309936,2024-02-09 00:00:00,1.0,ZMA,CSP,NaN,6139233834,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0,2024
139391,6028010260,6137309936,2023-11-30 00:00:00,1.0,ZMA,CSP,NaN,6137312894,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0,2023
139392,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0,100.0,0.0,2023
139393,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0,0.0,0.0,2023


**Изменим тип данных в столбце даты:**

In [60]:
final_inr['inr_date'] = final_inr['inr_date'].astype("datetime64[ns]")
final_inr

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside,below,above,year
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0,0.0,100.0,2023
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0,100.0,0.0,2023
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2020
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2020
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,6028010260,6137309936,2024-02-09 00:00:00,1.0,ZMA,CSP,NaN,6139233834,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0,2024
139391,6028010260,6137309936,2023-11-30 00:00:00,1.0,ZMA,CSP,NaN,6137312894,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0,2023
139392,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0,100.0,0.0,2023
139393,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0,0.0,0.0,2023


**Отсортируем по id и дате взятия МНО:**

In [61]:
final_inr = final_inr.sort_values(['peso_id','inr_date']).reset_index()
final_inr

,index,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside,below,above,year
0,24184,6026136442,6026188100,2016-12-01 00:00:00,3.2,LAB,CSP,3.0,6026188154,2016-12-01 12:08:43.621717,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2016
1,24268,6026136442,6026188100,2016-12-28 00:00:00,2.6,LAB,CSP,NaN,6026199130,NaN,NaN,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2016
2,24185,6026136442,6026188100,2017-01-23 00:00:00,3.7,LAB,CSP,2.0,6026206194,2017-01-23 11:10:56.575255,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0,0.0,100.0,2017
3,24186,6026136442,6026188100,2017-02-14 00:00:00,2.5,LAB,CSP,2.0,6026212948,2017-02-14 11:22:12.99316,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2017
4,24187,6026136442,6026188100,2017-04-03 00:00:00,3.0,LAB,CSP,2.0,6026221154,2017-04-03 10:51:08.935443,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,0.0,0.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139390,138944,6028057454,6140586056,2024-04-08 08:00:00,2.3,LAB,CSP,3.0,6140586104,2024-04-22 14:43:24.719202,return ''.gettext('Er zijn minder dan 2 opeenv...,NaN,NaN,2,2.5,3.5,0.0,100.0,0.0,2024
139391,138945,6028057456,6140586232,2024-04-22 08:00:00,2.5,LAB,CSP,3.0,6140586276,2024-04-22 14:43:24.697744,return ''.gettext('Er zijn minder dan 2 opeenv...,NaN,NaN,2,2.5,3.5,100.0,0.0,0.0,2024
139392,139235,6028057458,6140586428,2024-04-22 00:00:00,1.0,ZMA,CSP,NaN,6140586448,NaN,NaN,NaN,NaN,1,2.0,3.0,0.0,100.0,0.0,2024
139393,139002,6028057472,6140587670,2024-04-22 00:00:00,1.9,LAB,HCSP,NaN,6140587696,NaN,NaN,1.8,2.5,1,1.8,2.5,100.0,0.0,0.0,2024


In [62]:
(final_inr.loc[2,'inr_date']-final_inr.loc[1,'inr_date']).days

26

**Вычислим время нахождения в целевом диапазоне и общее время лечения:**

In [63]:
for i in range(len(final_inr)-1):
    if final_inr.loc[i,'peso_id'] == final_inr.loc[i+1,'peso_id']:
        final_inr.loc[i+1,'total_days'] = (final_inr.loc[i+1,'inr_date'] - final_inr.loc[i,'inr_date']).days
        if ((final_inr.loc[i,'below'] + final_inr.loc[i+1,'below'] == 200) or (final_inr.loc[i,'above'] + final_inr.loc[i+1,'above'] == 200)):
            final_inr.loc[i+1,'days_within_inr'] = 0
        elif final_inr.loc[i,'inside'] + final_inr.loc[i+1,'inside'] == 200:
            final_inr.loc[i+1,'days_within_inr'] = (final_inr.loc[i+1,'inr_date'] - final_inr.loc[i,'inr_date']).days
        elif    (final_inr.loc[i+1,'inr'] == final_inr.loc[i,'inr']):
            print(final_inr.loc[i,['peso_id','inr','inside','below','above','final_lower','final_upper']],final_inr.loc[i+1,['peso_id','inr','inside','below','above','final_lower','final_upper']])
        elif (final_inr.loc[i,'above'] + final_inr.loc[i+1,'below'] == 200) or (final_inr.loc[i,'below'] + final_inr.loc[i+1,'above'] == 200):
            final_inr.loc[i+1,'days_within_inr'] = (final_inr.loc[i+1,'final_upper'] - final_inr.loc[i,'final_lower'])/abs(final_inr.loc[i+1,'inr']-final_inr.loc[i,'inr'])*(final_inr.loc[i+1,'inr_date']-final_inr.loc[i,'inr_date']).days
        elif (final_inr.loc[i,'above'] + final_inr.loc[i+1,'inside'] == 200):
            final_inr.loc[i+1,'days_within_inr'] = (final_inr.loc[i+1,'final_upper'] - final_inr.loc[i+1,'inr'])/abs(final_inr.loc[i+1,'inr']-final_inr.loc[i,'inr'])*(final_inr.loc[i+1,'inr_date']-final_inr.loc[i,'inr_date']).days
        elif (final_inr.loc[i,'below'] + final_inr.loc[i+1,'inside'] == 200):
            final_inr.loc[i+1,'days_within_inr'] = (final_inr.loc[i+1,'inr'] - final_inr.loc[i,'final_lower'])/abs(final_inr.loc[i+1,'inr']-final_inr.loc[i,'inr'])*(final_inr.loc[i+1,'inr_date']-final_inr.loc[i,'inr_date']).days
        elif (final_inr.loc[i,'inside'] + final_inr.loc[i+1,'above'] == 200):
            final_inr.loc[i+1,'days_within_inr'] = (final_inr.loc[i+1,'final_upper'] - final_inr.loc[i,'inr'])/abs(final_inr.loc[i+1,'inr']-final_inr.loc[i,'inr'])*(final_inr.loc[i+1,'inr_date']-final_inr.loc[i,'inr_date']).days
        elif (final_inr.loc[i,'inside'] + final_inr.loc[i+1,'below'] == 200):
            final_inr.loc[i+1,'days_within_inr'] = (final_inr.loc[i,'inr'] - final_inr.loc[i,'final_lower'])/abs(final_inr.loc[i+1,'inr']-final_inr.loc[i,'inr'])*(final_inr.loc[i+1,'inr_date']-final_inr.loc[i,'inr_date']).days    

**Посмотрим, что получилось в интересующих нас столбцах:**

In [64]:
final_inr[['peso_id', 'inr','inr_date','final_lower', 'final_upper','below', 'inside', 'above', 'days_within_inr','total_days']].head()

,peso_id,inr,inr_date,final_lower,final_upper,below,inside,above,days_within_inr,total_days
0,6026136442,3.2,2016-12-01,2.5,3.5,0.0,100.0,0.0,NaN,NaN
1,6026136442,2.6,2016-12-28,2.5,3.5,0.0,100.0,0.0,27.000000,27.0
2,6026136442,3.7,2017-01-23,2.5,3.5,0.0,0.0,100.0,21.272727,26.0
3,6026136442,2.5,2017-02-14,2.5,3.5,0.0,100.0,0.0,18.333333,22.0
4,6026136442,3.0,2017-04-03,2.5,3.5,0.0,100.0,0.0,48.000000,48.0


**Создадим итоговую таблицу, отфильтровав только посещающих пациентов (не более 60 дней между визитами), и сгруппировав по годам:**

In [65]:
rosental = final_inr.query('total_days<=60').groupby(['year','method_cd'])[['days_within_inr','total_days']].sum()
rosental['TTR'] = rosental['days_within_inr']/rosental['total_days']*100
rosental = rosental.reset_index()
rosental['year'] = rosental['year'].dt.year
rosental

,year,method_cd,days_within_inr,total_days,TTR
0,2016,LAB,71.800000,148.0,48.513514
1,2017,LAB,22235.871410,37448.0,59.377995
2,2017,ZMA,0.000000,29.0,0.000000
3,2018,LAB,68706.089506,123176.0,55.778796
4,2018,ZMA,129.981112,409.0,31.780223
5,2019,LAB,188102.744479,328157.0,57.320961
6,2019,ZMA,1445.647765,3251.0,44.467787
7,2020,LAB,186386.492518,304713.0,61.167883
8,2020,ZMA,28188.458291,46525.0,60.587766
9,2021,LAB,254805.517576,414396.0,61.488411


**Построим график TTR по годам и методам измерения:**

In [66]:
fig = px.line(rosental, y = "TTR", color = "method_cd",x = "year", title='TTR у всех пациентов с разбивкой по годам и методам измерения')
fig.show()

**А теперь сгруппируем только по году:**

In [67]:
final = final_inr.query('total_days<=60').groupby('year')[['year','inside']].mean() #TTR
final = final.reset_index()
final['year'] = final['year'].dt.year
final.columns = ["Год", 'TTR']
final

,Год,TTR
0,2016,33.333333
1,2017,54.188635
2,2018,50.480000
3,2019,53.260183
4,2020,55.621160
5,2021,56.894768
6,2022,56.498209
7,2023,57.425095
8,2024,60.549974


**Пострим график для всех пациентов с разбивкой только по годам:**

In [70]:
fig = px.line(final, y = "TTR", x = "Год", title='TTR у всех пациентов с разбивкой по годам')
fig.show()

**Создадим для наглядности сводную таблицу TTR по годам для пациентов, сдающих анализ дома, и для пациентов с собственным прибором:**

In [69]:
final_inr.pivot_table(index ='method_cd', columns = 'year', values = 'inside', aggfunc = 'mean' ) #TTR pivot table

year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
method_cd,,,,,,,,,,,
LAB,0.0,50.0,53.571429,51.389566,49.432659,51.789632,54.826165,54.833021,54.701665,56.149089,59.381898
ZMA,NaN,NaN,NaN,0.000000,47.058824,36.842105,54.814815,61.407671,62.833100,59.021829,62.603878


## Выводы:
   1. Программа расчёта TTR методом Розенталя написана. При появлении новых данных достаточно её ещё раз запустить, чтобы получить актуальную информацию.
   2. Результаты анализа данных сведены в таблицы и визулизированы при помощи графиков. 
   3. Из графиков и таблиц следует, что:
      - у пациентов, сдающих анализы в лаборатрии, есть данные начиная с 2014 года;
      - у пациентов, сдающих анализы в лаборатрии, есть данные начиная с 2017 года;
      - пациенты, сдающие анализы в лаборатрии, в 2017ом году имели более высокий TTR, но после 2020 года самоизмерямые пациенты обогнали первых по этому параметру;
      - независимо от способа сдачи МНО, отчётливо виден рост TTR из года в год.